In [1]:
from pymongo import MongoClient
import datetime
import calendar
import pandas as pd
import numpy as np
import Config
import time
pd.set_option('display.max_rows', None)
client = MongoClient(Config.DB_Hostname,Config.DB_Port)

def Nifty_Startegy_Variants():
    position_multiplier_string={"1/1|3/2|11/10|1/3":[1,1.5,1.1,1/3],"1/1|3/2|11/10|1/2":[1,1.5,1.1,0.5]}
    move_multiplier_string={"5/4|1/1|3/2":[1.25,1,1.5],"11/10|1/1|3/2":[1.1,1,1.5]}
    profit_multiplier_string={"1/1|1/1|3/2":[1,1,1.5],"1/1|1/1|2/1":[1,1,2],"1/1|1/2|2/1":[1,0.5,2],"1/1|3/2|3/1":[1,1.5,3]}
    week_parameter=[1,0]
    strategy_type_parameter=[1,2]
    strategy_version_parameter=[1,2]
    position_ratios=[0.5,0.6,2/3]
    strat_summary={"underlying":[],"strategy_variant":[],"position_multiplier_string":[],"position_multiplier":[],
               "abstinence_multiplier":[],"straight_abstinence_multiplier":[],"straight_reversal_multiplier":[],
               "move_multiplier_string":[],"extreme_move_multiplier":[],"directional_move_multiplier":[],
               "initial_move_multiplier":[],"profit_multiplier_string":[],"take_profit_multiplier":[],
               "hedge_manage_multiplier":[],"stop_loss_multiplier":[],"week_parameter":[],
               "strategy_type_parameter":[],"strategy_version_parameter":[],"position_ratio":[]}

    counter = 1
    for (pos_str,pos_val) in position_multiplier_string.items():
        for (mov_str,mov_val) in move_multiplier_string.items():
            for (prof_str,prof_val) in profit_multiplier_string.items():
                for week in week_parameter:
                    for stype in strategy_type_parameter:
                        for sver in strategy_version_parameter:
                            for p_ratio in position_ratios:
                                strat_summary["underlying"].append("NIFTY")
                                strat_summary["strategy_variant"].append(counter)

                                strat_summary["position_multiplier_string"].append(pos_str)
                                strat_summary["position_multiplier"].append(pos_val[0])
                                strat_summary["abstinence_multiplier"].append(pos_val[1])
                                strat_summary["straight_abstinence_multiplier"].append(pos_val[2])
                                strat_summary["straight_reversal_multiplier"].append(pos_val[3])

                                strat_summary["move_multiplier_string"].append(mov_str)
                                strat_summary["extreme_move_multiplier"].append(mov_val[0])
                                strat_summary["directional_move_multiplier"].append(mov_val[1])
                                strat_summary["initial_move_multiplier"].append(mov_val[2])

                                strat_summary["profit_multiplier_string"].append(prof_str)
                                strat_summary["take_profit_multiplier"].append(prof_val[0])
                                strat_summary["hedge_manage_multiplier"].append(prof_val[1])
                                strat_summary["stop_loss_multiplier"].append(prof_val[2])

                                strat_summary["week_parameter"].append(week)
                                strat_summary["strategy_type_parameter"].append(stype)
                                strat_summary["strategy_version_parameter"].append(sver)
                                strat_summary["position_ratio"].append(p_ratio)


                                counter+=1
    df=pd.DataFrame(strat_summary)
    print(client.Strategy.Strategy_Variants.delete_many({"underlying":"NIFTY"}).deleted_count)
    print(len(client.Strategy.Strategy_Variants.insert_many(df.to_dict('records')).inserted_ids))

def LastThInMonth(year, month):
    
    # Create a datetime.date for the last day of the given month
    daysInMonth = calendar.monthrange(year, month)[1]   # Returns (month, numberOfDaysInMonth)
    dt = datetime.date(year, month, daysInMonth)

    # Back up to the most recent Thursday
    offset = 4 - dt.isoweekday()
    if offset > 0: offset -= 7                          # Back up one week if necessary
    dt += datetime.timedelta(offset)                    # dt is now date of last Th in month
    
    return dt

def _days_to_expiry(underlying, start_date,end_date):

    #Beware of 3 or more consequtive holidays
    start_date=datetime.datetime.strptime(start_date,"%Y-%m-%d")
    end_date=datetime.datetime.strptime(end_date,"%Y-%m-%d")

    holidays=pd.DataFrame(client.Strategy.Holidays.find()).sort_values('date')
    holidays.drop(columns=["_id"],inplace=True)

    date=start_date
    date_string=[]
    current_week=[]
    next_week=[]

    while date != end_date:
        w=date.weekday()
        if w not in [5,6]:
            date_string.append(datetime.datetime.strftime(date,"%Y-%m-%d"))
        date+=datetime.timedelta(days = 1)

    extra_days=pd.DataFrame(client.Strategy.Working_Holidays.find()).sort_values('date')

    for date in extra_days.date:
        date_string.append(date)

    date_string=sorted(date_string)


    df=pd.DataFrame()
    df["date"]=date_string
    df["Holiday"]=np.where(df.date.isin(holidays.date),1,0)
    df["week_day"]=df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%d").weekday())

    if underlying == "NIFTY":
        df["expiry"]=np.where(df.week_day==3,1,0)
    else:
        monthly_expiry = []
        for year in list(range(2015, end_date.year + 1)):
            for month in range(1, 13):
                monthly_expiry.append(datetime.datetime.strftime(LastThInMonth(year, month), "%Y-%m-%d"))

        faulty_wednesday_expiry = [datetime.datetime.strftime(datetime.datetime.strptime(thursday_expiry, "%Y-%m-%d") - datetime.timedelta(days = 1), "%Y-%m-%d") for thursday_expiry in monthly_expiry]
        df["expiry"]=np.where(df.week_day==2,1,0)
        df.expiry = np.where(df.date.isin(faulty_wednesday_expiry), 0, df.expiry)
        df.expiry = np.where(df.date.isin(monthly_expiry), 1, df.expiry)

    week_day=df.week_day.tolist()
    Holiday=df.Holiday.tolist()
    date_string=df.date.tolist()
    normal_expiry=df.expiry.tolist()
    expiry_day=[]

    for i,date in enumerate(date_string):
        if normal_expiry[i]==1:
            if Holiday[i]==0:
                expiry_day.append(date)
            elif Holiday[i-1]==0:
                expiry_day.append(date_string[i-1])
            else:
                expiry_day.append(date_string[i-2])
    df["actual_expiry"]=np.where(df.date.isin(expiry_day),1,0)
    expiry_day=df.actual_expiry.tolist()

    current_week=[]
    next_week=[]

    for i,date in enumerate(date_string):

        if Holiday[i]==1:
            current_week.append(-1)
            next_week.append(-1)
            continue

        expiry_count=0
        curr_temp=0
        next_temp=0

        try:
            for j,date_ in enumerate(date_string[i:]):

                if Holiday[i+j]==1:
                    continue

                next_temp+=1        
                if expiry_count<1:
                    curr_temp+=1

                if expiry_day[i+j]==1:
                    expiry_count+=1

                if expiry_count==2:
                    break
        except Exception as e:
            print(e)
            current_week.append(np.nan)
            next_week.append(np.nan)

        current_week.append(curr_temp)
        next_week.append(next_temp)


    df["current_week"]=current_week
    df["next_week"]=next_week

    df=df[df.Holiday==0]
    df=df[['date','current_week','next_week']]
    df.sort_values('date',inplace=True)
    df["underlying"]=underlying
    return df

def Days_To_Expiry(start_date, end_date):
    
    Nifty_df = _days_to_expiry("NIFTY", start_date, end_date)
    Banknifty_df = _days_to_expiry("BANKNIFTY", start_date, end_date)
    Banknifty_df = pd.concat([Nifty_df[Nifty_df.date <= "2023-09-01"], Banknifty_df[Banknifty_df.date > "2023-09-01"]])
    Banknifty_df.underlying = "BANKNIFTY"
    
    df = pd.concat([Nifty_df, Banknifty_df]).sort_values(['date', 'underlying'])
    
    print(client.Strategy.Days_To_Expiry.delete_many({}).deleted_count)
    print(len(client.Strategy.Days_To_Expiry.insert_many(df.to_dict('records')).inserted_ids))

In [2]:
Days_To_Expiry("2015-01-09", "2024-12-31")

4922
4920


In [ ]:
import matplotlib.pyplot as plt
from pymongo import MongoClient
import datetime
import calendar
import pandas as pd
import numpy as np
import Config
import time
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
client = MongoClient(Config.DB_Hostname,Config.DB_Port)

pred = pd.DataFrame(client.Live_Trading.Live_Strategies.find()).drop(columns = ['_id']).sort_values('date')
res = pd.DataFrame(client.Strategy.Live_Strategies.find()).drop(columns = ['_id']).sort_values('date')
merged = pd.merge(pred.set_index(['date', 'strategy_variant']), res[['date', 'strategy_variant', 'returns']].set_index(['date', 'strategy_variant']),left_index=True, right_index=True).reset_index().groupby('date').returns.mean()
merged += 1
merged.cumprod().plot(figsize = (20,10))

In [ ]:
res_ = pd.DataFrame(client.Strategy.Live_Strategies.find()).sort_values('date')
res_ = (1 + res_[res_.prediction == 1].groupby('date').returns.mean())
res_[res_.index>="2023-06-05"].cumprod().plot(figsize = (20,10))

In [ ]:
df = pd.DataFrame(index = sorted(set(res_[res_.index>="2023-06-05"].index.tolist() + merged.index.tolist())))
df["filtered"] = merged
df["unfiltered"] = res_[res_.index>="2023-06-05"]
df.filtered = np.where(df.filtered.isna(), df.unfiltered, df.filtered)
df.fillna(1, inplace = True)
df.filtered = np.where(df.filtered == 1, df.filtered, df.filtered + 0.)
df.filtered.cumprod().plot(figsize = (20,10))

In [ ]:
res_ = pd.DataFrame(client.Strategy.Live_Strategies.find()).sort_values('date')
ret_ = res_[res_.prediction == 1].groupby('date').returns.mean()
res_ = (1 + ret_)
res_.sort_index().cumprod().plot(figsize = (20, 10))

In [ ]:
res_ = pd.DataFrame(client.Strategy.Live_Strategies.find()).sort_values('date').drop(columns = ['_id'])
res_ = res_[res_.prediction == 1]
res_.returns = np.sign(res_.returns)
res_.returns[res_.returns==0]= 1
res_.returns[res_.returns==-1]= 0
res_.groupby('date').returns.mean().rolling(5).mean().plot(figsize = (20,10))

In [ ]:
res_ = pd.DataFrame(client.Strategy.Live_Strategies.find()).sort_values('date')
ret_ = res_[res_.prediction == 1].groupby('date').returns.mean()
res_ = (1 + ret_)
fig, ax = plt.subplots(figsize = (20, 10))
ax.plot(ret_.sort_index().rolling(5).mean().dropna().tolist())
ax.axhline(y=0, color = "black")

In [ ]:
res_ = pd.DataFrame(client.Strategy.Live_Strategies.find()).sort_values('date')
res_ = (1 + res_[res_.prediction == 1].groupby('date').returns.mean())
np.log(res_.cumprod()).plot(figsize = (20,10))

In [ ]:
import matplotlib.pyplot as plt
from pymongo import MongoClient
import datetime
import calendar
import pandas as pd
import numpy as np
import Config
import time
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

client = MongoClient(Config.DB_Hostname,Config.DB_Port)

index_ts = pd.DataFrame(client.Live_Trading.Index_Tracker.find()).drop(columns = ['_id', 'underlying'])
index_ts = index_ts[index_ts.timestamp > 1705290300].sort_values('timestamp')
index_ts.timestamp = range(len(index_ts))
index_ts.set_index('timestamp', inplace = True)

In [ ]:


def generate_correlation(lag, quantile, index_ts, verbose = False):
    index_ts_lagged = index_ts.shift(lag)
    index_ts_lagged.rename(columns={'price':'price_lagged'}, inplace=True)
    index_ts_lead = index_ts.shift(-10)
    index_ts_lead.rename(columns={'price':'price_lead'}, inplace=True)
    merged_index = pd.merge(index_ts, index_ts_lagged, left_index=True, right_index=True)
    merged_index = pd.merge(merged_index, index_ts_lead, left_index=True, right_index=True).dropna()

    merged_index["Independent"] = merged_index.price - merged_index.price_lagged
    merged_index["Dependent"] = merged_index.price_lead - merged_index.price
    
    if verbose:
        print(abs(merged_index.Independent).quantile(quantile), len(merged_index[abs(merged_index.Independent)>abs(merged_index.Independent).quantile(quantile)]))
    
    merged_index = merged_index[abs(merged_index.Independent)>abs(merged_index.Independent).quantile(quantile)]
    
    if verbose:
        plt.scatter(merged_index.Independent, merged_index.Dependent)
    indicator = merged_index.Independent * merged_index.Dependent
    
    return len(indicator[indicator<=0])/len(indicator), len(indicator)

In [ ]:
quantile = []
lag = []
correl = []
num_samples = []
for l in range(1,121):
    for q in [0.9, 0.95, 0.99, 0.991, 0.992, 0.993, 0.994, 0.995, 0.996, 0.997, 0.998, 0.999]:
        lag.append(l)
        quantile.append(q)
        c, n = generate_correlation(l, q, index_ts)
        correl.append(c)
        num_samples.append(n)
        
df = pd.DataFrame({"q": quantile, "lag" : lag, "correlation" : correl, "num_samples" : num_samples})


In [ ]:
df[df.correlation>=0.65].sort_values('num_samples', ascending=False)

In [ ]:
generate_correlation(84, 0.99, index_ts, verbose=True)

In [ ]:
20.052999999999113*1e4/22100

In [ ]:
1705031100 + 24*3600*3

In [ ]:
res_